In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

import sklearn
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# Import datasets
!wget -O dataset_combined_uncleaned.csv https://raw.githubusercontent.com/godhi06/TA/master/data/preprocessing/data/df_ori.csv

!wget -O data_pre_1.csv https://raw.githubusercontent.com/godhi06/TA/master/data/preprocessing/data/df_pre1.csv
!wget -O data_pre_1_stemmed.csv https://raw.githubusercontent.com/godhi06/TA/master/data/preprocessing/data/pre1_stemmed.csv
!wget -O data_pre_1_lemma.csv https://raw.githubusercontent.com/godhi06/TA/master/data/preprocessing/data/pre1_lemma.csv 


!wget -O data_pre_2.csv https://raw.githubusercontent.com/godhi06/TA/master/data/preprocessing/data/df_pre2.csv
!wget -O data_pre_2_stemmed.csv https://raw.githubusercontent.com/godhi06/TA/master/data/preprocessing/data/pre2_stemmed.csv
!wget -O data_pre_2_lemmatized.csv https://raw.githubusercontent.com/godhi06/TA/master/data/preprocessing/data/pre2_lemma.csv


dfRaw = pd.read_csv('dataset_combined_uncleaned.csv', index_col=0) # combined uncleaned

dfPre1 = pd.read_csv('data_pre_1.csv', index_col=0) # with stopwordsq
dfPre1_stemmed = pd.read_csv('data_pre_1_stemmed.csv', index_col=0) # without stopwords; stemmed
dfPre1_lemma = pd.read_csv('data_pre_1_lemma.csv', index_col=0) # without stopwords; lemmatized

dfPre2 = pd.read_csv('data_pre_2.csv', index_col=0) # without stopwords; without stem or lemma
dfPre2_stemmed = pd.read_csv('data_pre_2_stemmed.csv', index_col=0) # without stopwords; stemmed
dfPre2_lemma = pd.read_csv('data_pre_2_lemmatized.csv', index_col=0) # without stopwords; lemmatized

--2020-06-20 14:18:16--  https://raw.githubusercontent.com/godhi06/TA/master/data/preprocessing/data/df_ori.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 222570 (217K) [text/plain]
Saving to: ‘dataset_combined_uncleaned.csv’

dataset_combined_un 100%[===================>] 217.35K  --.-KB/s    in 0.05s   

2020-06-20 14:18:17 (4.10 MB/s) - ‘dataset_combined_uncleaned.csv’ saved [222570/222570]

--2020-06-20 14:18:18--  https://raw.githubusercontent.com/godhi06/TA/master/data/preprocessing/data/df_pre1.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response.

In [ ]:
print("Banyak dfRaw : ", len(dfRaw))
print("\n")
print("Banyak dfPre1 : ", len(dfPre1))
print("Banyak dfPre1_stemmed : ", len(dfPre1_stemmed))
print("Banyak dfPre1_lemma : ", len(dfPre1_lemma))
print("\n")
print("Banyak dfPre2 : ", len(dfPre2))
print("Banyak dfPre2_stemmed : ", len(dfPre2_stemmed))
print("Banyak dfPre2_lemma : ", len(dfPre2_lemma))

Banyak dfRaw :  646


Banyak dfPre1 :  646
Banyak dfPre1_stemmed :  646
Banyak dfPre1_lemma :  646


Banyak dfPre2 :  646
Banyak dfPre2_stemmed :  646
Banyak dfPre2_lemma :  646


In [ ]:
from sklearn.model_selection import train_test_split
# Split dfRaw
X = dfRaw['text'].values
y = dfRaw['label'].values

X_train_raw, X_test_raw, y_train_raw, y_test_raw = train_test_split(X, y, test_size=0.3, random_state=42)


In [ ]:
print("dfRaw (combined & unclean)")
print("Banyak data train : ", len(X_train_raw))
print("Banyak data test : ", len(X_test_raw))
total_raw = len(X_train_raw) + len(X_test_raw)
print("Total : " , total_raw, ". Apakah sesuai ?", total_raw == len(dfRaw))

dfRaw (combined & unclean)
Banyak data train :  452
Banyak data test :  194
Total :  646 . Apakah sesuai ? True


In [ ]:
dfTrain_Raw = dfRaw[dfRaw.text.isin(X_train_raw)]
dfTrain_Raw

,text,label
0,I've had major depressive disorder for many y...,1
1,One of my friends lost his brother to suicide...,1
3,I was recently diagnosed with GAD (Generalize...,1
4,"After 4 years my hair is finally healed, & lif...",0
5,Does anyone know when the proper time is to g...,1
...,...,...
638,"No matter how difficult life can be, I try to ...",0
639,"I was diagnosed with PTSD, STPD, and MDD when...",1
640,"Just carved DECAY into my lower thigh, near m...",1
643,"In his videos about self injury, not only doe...",1


In [ ]:
dfTest_Raw = dfRaw[dfRaw.text.isin(X_test_raw)]
dfTest_Raw

,text,label
2,My long distance boyfriend sent me flowers for...,0
6,Military son brings his single father to tears...,0
7,This was yesterdayâs sunrise isnât it beau...,0
10,I thought I'd hit my limit a few months ago w...,1
11,"Can you force yourself to have anxiety, just ...",1
...,...,...
627,I posted on here recently about how I was cho...,1
633,In honor of my twin sister all these new books...,0
641,I wish I could shake this. I don't laugh or j...,1
642,Hereâs a picture of grass smiling to brighte...,0


In [ ]:
dfTrain_pre1 = dfPre1.iloc[dfTrain_Raw.index]
dfTrain_pre1

,text,label
0,i have had major depressive disorder for many ...,1
1,one of my friends lost his brother to suicide ...,1
3,i was recently diagnosed with gad generalized ...,1
4,after years my hair is finally healed life is ...,0
5,does anyone know when the proper time is to go...,1
...,...,...
638,no matter how difficult life can be i try to k...,0
639,i was diagnosed with ptsd stpd and mdd when i ...,1
640,just carved decay into my lower thigh near my ...,1
643,in his videos about self injury not only does ...,1


In [ ]:
dfTest_pre1 = dfPre1.iloc[dfTest_Raw.index]
dfTest_pre1

,text,label
2,my long distance boyfriend sent me flowers for...,0
6,military son brings his single father to tears...,0
7,this was yesterdays sunrise isnt it beautiful ...,0
10,i thought i would hit my limit a few months ag...,1
11,can you force yourself to have anxiety just be...,1
...,...,...
627,i posted on here recently about how i was chok...,1
633,in honor of my twin sister all these new books...,0
641,i wish i could shake this i do not laugh or jo...,1
642,heres a picture of grass smiling to brighten u...,0


In [ ]:
dfTrain_Pre1stem = dfPre1_stemmed.iloc[dfTrain_Raw.index]
dfTrain_Pre1stem

,text,label
0,i have had major depress disord for mani year ...,1
1,one of my friend lost hi brother to suicid yea...,1
3,i wa recent diagnos with gad gener anxieti dis...,1
4,after year my hair is final heal life is also ...,0
5,doe anyon know when the proper time is to go t...,1
...,...,...
638,no matter how difficult life can be i tri to k...,0
639,i wa diagnos with ptsd stpd and mdd when i wa ...,1
640,just carv decay into my lower thigh near my ca...,1
643,in hi video about self injuri not onli doe he ...,1


In [ ]:
dfTest_Pre1stem = dfPre1_stemmed.iloc[dfTest_Raw.index]
dfTest_Pre1stem

,text,label
2,my long distanc boyfriend sent me flower for n...,0
6,militari son bring hi singl father to tear wit...,0
7,thi wa yesterday sunris isnt it beauti i love ...,0
10,i thought i would hit my limit a few month ago...,1
11,can you forc yourself to have anxieti just bec...,1
...,...,...
627,i post on here recent about how i wa choke by ...,1
633,in honor of my twin sister all these new book ...,0
641,i wish i could shake thi i do not laugh or jok...,1
642,here a pictur of grass smile to brighten up yo...,0


In [ ]:
dfTrain_Pre1lem = dfPre1_lemma.iloc[dfTrain_Raw.index]
dfTrain_Pre1lem

,text,label
0,i have had major depressive disorder for many ...,1
1,one of my friend lost his brother to suicide y...,1
3,i wa recently diagnosed with gad generalized a...,1
4,after year my hair is finally healed life is a...,0
5,doe anyone know when the proper time is to go ...,1
...,...,...
638,no matter how difficult life can be i try to k...,0
639,i wa diagnosed with ptsd stpd and mdd when i w...,1
640,just carved decay into my lower thigh near my ...,1
643,in his video about self injury not only doe he...,1


In [ ]:
dfTest_Pre1lem = dfPre1_lemma.iloc[dfTest_Raw.index]
dfTest_Pre1lem

,text,label
2,my long distance boyfriend sent me flower for ...,0
6,military son brings his single father to tear ...,0
7,this wa yesterday sunrise isnt it beautiful i ...,0
10,i thought i would hit my limit a few month ago...,1
11,can you force yourself to have anxiety just be...,1
...,...,...
627,i posted on here recently about how i wa choke...,1
633,in honor of my twin sister all these new book ...,0
641,i wish i could shake this i do not laugh or jo...,1
642,here a picture of grass smiling to brighten up...,0


In [ ]:
dfTrain_Pre2 = dfPre2.iloc[dfTrain_Raw.index]
dfTrain_Pre2

,text,label,wc_count
0,major depressive disorder many years got point...,1,56
1,one friends lost brother suicide years ago sti...,1,48
3,recently diagnosed gad generalized anxiety dis...,1,51
4,years hair finally healed life also getting mu...,0,31
5,anyone know proper time go hospital ive super ...,1,27
...,...,...,...
638,matter difficult life try keep sense humor pos...,0,35
639,diagnosed ptsd stpd mdd twenty two would like ...,1,16
640,carved decay lower thigh near calf pretty deep...,1,58
643,videos self injury almost completely ignore fo...,1,119


In [ ]:
dfTest_Pre2 = dfPre2.iloc[dfTest_Raw.index]
dfTest_Pre2

,text,label,wc_count
2,long distance boyfriend sent flowers particula...,0,12
6,military son brings single father tears heartw...,0,9
7,yesterdays sunrise isnt beautiful love sun rises,0,7
10,thought would hit limit months ago tried kill ...,1,77
11,force anxiety want admit like thoughts also oc...,1,16
...,...,...,...
627,posted recently choked student work work stude...,1,78
633,honor twin sister new books got donated local ...,0,21
641,wish could shake laugh joke life voice anymore...,1,22
642,heres picture grass smiling brighten day,0,6


In [ ]:
dfTrain_Pre2stem = dfPre2_stemmed.iloc[dfTrain_Raw.index]
dfTrain_Pre2stem

,text,label
0,major depress disord mani year got point somet...,1
1,one friend lost brother suicid year ago still ...,1
3,recent diagnos gad gener anxieti disord hasnt ...,1
4,year hair final heal life also get much better...,0
5,anyon know proper time go hospit ive super dep...,1
...,...,...
638,matter difficult life tri keep sens humor posi...,0
639,diagnos ptsd stpd mdd twenti two would like go...,1
640,carv decay lower thigh near calf pretti deep a...,1
643,video self injuri almost complet ignor form se...,1


In [ ]:
dfTest_Pre2stem = dfPre2_stemmed.iloc[dfTest_Raw.index]
dfTest_Pre2stem

,text,label
2,long distanc boyfriend sent flower particular ...,0
6,militari son bring singl father tear heartwarm...,0
7,yesterday sunris isnt beauti love sun rise,0
10,thought would hit limit month ago tri kill com...,1
11,forc anxieti want admit like thought also ocd ...,1
...,...,...
627,post recent choke student work work student au...,1
633,honor twin sister new book got donat local lib...,0
641,wish could shake laugh joke life voic anymor a...,1
642,here pictur grass smile brighten day,0


In [ ]:
dfTrain_Pre2lem = dfPre2_lemma.iloc[dfTrain_Raw.index]
dfTrain_Pre2lem

,text,label
0,major depressive disorder many year got point ...,1
1,one friend lost brother suicide year ago still...,1
3,recently diagnosed gad generalized anxiety dis...,1
4,year hair finally healed life also getting muc...,0
5,anyone know proper time go hospital ive super ...,1
...,...,...
638,matter difficult life try keep sense humor pos...,0
639,diagnosed ptsd stpd mdd twenty two would like ...,1
640,carved decay lower thigh near calf pretty deep...,1
643,video self injury almost completely ignore for...,1


In [ ]:
dfTest_Pre2lem = dfPre2_lemma.iloc[dfTest_Raw.index]
dfTest_Pre2lem

,text,label
2,long distance boyfriend sent flower particular...,0
6,military son brings single father tear heartwa...,0
7,yesterday sunrise isnt beautiful love sun rise,0
10,thought would hit limit month ago tried kill c...,1
11,force anxiety want admit like thought also ocd...,1
...,...,...
627,posted recently choked student work work stude...,1
633,honor twin sister new book got donated local l...,0
641,wish could shake laugh joke life voice anymore...,1
642,here picture grass smiling brighten day,0


In [ ]:
from google.colab import files

dfTrain_Raw.to_csv('dfTrain_Raw.csv')
dfTest_Raw.to_csv('dfTest_Raw.csv')

dfTrain_pre1.to_csv('dfTrain_pre1.csv')
dfTest_pre1.to_csv('dfTest_pre1.csv')

dfTrain_Pre1stem.to_csv('dfTrain_Pre1stem.csv')
dfTest_Pre1stem.to_csv('dfTest_Pre1stem.csv')

dfTrain_Pre1lem.to_csv('dfTrain_Pre1lem.csv')
dfTest_Pre1lem.to_csv('dfTest_Pre1lem.csv')

dfTrain_Pre2.to_csv('dfTrain_Pre2.csv')
dfTest_Pre2.to_csv('dfTest_Pre2.csv')

dfTrain_Pre2stem.to_csv('dfTrain_Pre2stem.csv')
dfTest_Pre2stem.to_csv('dfTest_Pre2stem.csv')

dfTrain_Pre2lem.to_csv('dfTrain_Pre2lem.csv')
dfTest_Pre2lem.to_csv('dfTest_Pre2lem.csv')




files.download("dfTrain_Raw.csv")
files.download("dfTest_Raw.csv")

files.download("dfTrain_pre1.csv")
files.download("dfTest_pre1.csv")

files.download("dfTrain_Pre1stem.csv")
files.download("dfTest_Pre1stem.csv")

files.download("dfTrain_Pre1lem.csv")
files.download("dfTest_Pre1lem.csv")

files.download("dfTrain_Pre2.csv")
files.download("dfTest_Pre2.csv")

files.download("dfTrain_Pre2stem.csv")
files.download("dfTest_Pre2stem.csv")

files.download("dfTrain_Pre2lem.csv")
files.download("dfTest_Pre2lem.csv")